In [47]:
#Importing Libararies
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
pd.set_option('display.precision',5)
import time

In [48]:
# Loading Dataset
start_time = time.time()
users=pd.read_csv("./Dataset/users.csv")
products=pd.read_csv("./Dataset/products.csv")
inventory_items=pd.read_csv("./Dataset/inventory_items.csv")
orders=pd.read_csv("./Dataset/orders.csv")
order_items=pd.read_csv("./Dataset/order_items.csv")
events=pd.read_csv("./Dataset/events.csv")
dist_centers=pd.read_csv("./Dataset/distribution_centers.csv")
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

12.438841104507446


# ORDERS MERGE (orders & order_items)

In [49]:
orders.shape

(125226, 9)

In [50]:
orders.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1
3,89,65,Cancelled,F,2020-08-13 09:58:00+00:00,NaN,NaN,NaN,1
4,102,76,Cancelled,F,2023-01-17 08:17:00+00:00,NaN,NaN,NaN,2


In [51]:
for column in orders.columns:
    unique_values = orders[column].nunique()
    print(f"{column}: {unique_values}")

order_id: 125226
user_id: 80044
status: 5
gender: 2
created_at: 117373
returned_at: 12478
shipped_at: 79123
delivered_at: 43187
num_of_item: 4


In [52]:
orders.loc[orders['user_id']==28215] 

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
13613,35223,28215,Complete,F,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
28380,35222,28215,Processing,F,2022-08-24 13:23:00+00:00,NaN,NaN,NaN,2


In [53]:
order_items.shape

(181759, 11)

In [54]:
181759-125226

56533

In [55]:
order_items.head()

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,0.02
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaN,NaN,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaN,NaN,0.02


In [56]:
for column in order_items.columns:
    unique_values = order_items[column].nunique()
    print(f"{column}: {unique_values}")

id: 181759
order_id: 125226
user_id: 80044
product_id: 29046
inventory_item_id: 181759
status: 5
created_at: 181526
shipped_at: 79123
delivered_at: 43187
returned_at: 12478
sale_price: 4190


In [57]:
order_items.loc[order_items['user_id']==28215] 

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02
24869,51226,35223,28215,10641,138242,Complete,2023-12-06 10:12:51+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,16.00
33186,51222,35222,28215,7101,138230,Processing,2022-08-27 11:09:05+00:00,NaN,NaN,NaN,19.95
34158,51227,35223,28215,5268,138245,Complete,2023-12-07 10:02:18+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,19.99
141747,51225,35223,28215,13268,138240,Complete,2023-12-04 12:42:42+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,78.00
171932,51223,35222,28215,8039,138232,Processing,2022-08-28 12:18:49+00:00,NaN,NaN,NaN,169.00


In [58]:
orders_final = pd.merge(order_items, orders, 
                          left_on=['order_id', 'user_id'],
                          right_on=['order_id','user_id'],
                          how='left', 
                          suffixes=('_item', '_order'))
orders_final.head()

,id,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,shipped_at_item,delivered_at_item,returned_at_item,sale_price,status_order,gender,created_at_order,returned_at_order,shipped_at_order,delivered_at_order,num_of_item
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02,Cancelled,F,2023-05-07 07:33:00+00:00,NaN,NaN,NaN,1
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,0.02,Complete,F,2023-03-14 03:59:00+00:00,NaN,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,1
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02,Complete,F,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaN,NaN,0.02,Shipped,F,2023-12-23 00:17:00+00:00,NaN,2023-12-24 16:44:00+00:00,NaN,4
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaN,NaN,0.02,Shipped,F,2022-06-17 17:21:00+00:00,NaN,2022-06-19 19:29:00+00:00,NaN,3


In [59]:
orders_final.loc[orders_final['user_id']==28215] 

,id,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,shipped_at_item,delivered_at_item,returned_at_item,sale_price,status_order,gender,created_at_order,returned_at_order,shipped_at_order,delivered_at_order,num_of_item
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02,Complete,F,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
24869,51226,35223,28215,10641,138242,Complete,2023-12-06 10:12:51+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,16.00,Complete,F,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
33186,51222,35222,28215,7101,138230,Processing,2022-08-27 11:09:05+00:00,NaN,NaN,NaN,19.95,Processing,F,2022-08-24 13:23:00+00:00,NaN,NaN,NaN,2
34158,51227,35223,28215,5268,138245,Complete,2023-12-07 10:02:18+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,19.99,Complete,F,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
141747,51225,35223,28215,13268,138240,Complete,2023-12-04 12:42:42+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,78.00,Complete,F,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
171932,51223,35222,28215,8039,138232,Processing,2022-08-28 12:18:49+00:00,NaN,NaN,NaN,169.00,Processing,F,2022-08-24 13:23:00+00:00,NaN,NaN,NaN,2


In [60]:
orders_final.shape

(181759, 18)

In [61]:
orders_final['shipped_at_order'] = orders_final['shipped_at_order'].replace('NaT', np.nan)

In [62]:
orders_final.dtypes

id                      int64
order_id                int64
user_id                 int64
product_id              int64
inventory_item_id       int64
status_item            object
created_at_item        object
shipped_at_item        object
delivered_at_item      object
returned_at_item       object
sale_price            float64
status_order           object
gender                 object
created_at_order       object
returned_at_order      object
shipped_at_order       object
delivered_at_order     object
num_of_item             int64
dtype: object

# Code to check whether the ccolumns values are same or not

# DROPPING COLUMNS

In [63]:
orders_final=orders_final.drop(columns=['status_order','returned_at_item','shipped_at_item','delivered_at_item'])
orders_final.head()

,id,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,sale_price,gender,created_at_order,returned_at_order,shipped_at_order,delivered_at_order,num_of_item
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,0.02,F,2023-05-07 07:33:00+00:00,NaN,NaN,NaN,1
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,0.02,F,2023-03-14 03:59:00+00:00,NaN,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,1
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,0.02,F,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,0.02,F,2023-12-23 00:17:00+00:00,NaN,2023-12-24 16:44:00+00:00,NaN,4
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,0.02,F,2022-06-17 17:21:00+00:00,NaN,2022-06-19 19:29:00+00:00,NaN,3


# FINAL ORDERS DATASET WITH PRODUCTS (MERGE)

In [64]:
products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


In [65]:
products.shape

(29120, 9)

In [66]:
orders_products=pd.merge(orders_final,products, left_on='product_id',right_on='id',suffixes=('_orderedproducts', '_product'))
orders_products.head()

,id_orderedproducts,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,sale_price,gender,created_at_order,...,num_of_item,id_product,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,0.02,F,2023-05-07 07:33:00+00:00,...,1,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,0.02,F,2023-03-14 03:59:00+00:00,...,1,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,0.02,F,2023-12-03 13:23:00+00:00,...,4,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,0.02,F,2023-12-23 00:17:00+00:00,...,4,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,0.02,F,2022-06-17 17:21:00+00:00,...,3,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1


In [67]:
orders_products.loc[orders_products['user_id']==28215]

,id_orderedproducts,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,sale_price,gender,created_at_order,...,num_of_item,id_product,cost,category,name,brand,retail_price,department,sku,distribution_center_id
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,0.02,F,2023-12-03 13:23:00+00:00,...,4,14235,0.00830,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
25212,51226,35223,28215,10641,138242,Complete,2023-12-06 10:12:51+00:00,16.00,F,2023-12-03 13:23:00+00:00,...,4,10641,7.96800,Intimates,Maidenform Womens Dream Front Close Lace T-Bac...,Maidenform,16.00,Women,C95D62C68196B2D0C1C1DE8C7EEB6D50,9
33377,51222,35222,28215,7101,138230,Processing,2022-08-27 11:09:05+00:00,19.95,F,2022-08-24 13:23:00+00:00,...,2,7101,7.99995,Skirts,Black Shiny Liquid Mini Skirt Elastic Waist Band,FineBrandShop,19.95,Women,B33128CB0089003DDFB5199E1B679652,7
34789,51227,35223,28215,5268,138245,Complete,2023-12-07 10:02:18+00:00,19.99,F,2023-12-03 13:23:00+00:00,...,4,5268,11.31434,Pants & Capris,Wearables womens ruffle pocket capri pants,Wearables,19.99,Women,02A3C7FB3F489288AE6942498498DB20,3
141948,51225,35223,28215,13268,138240,Complete,2023-12-04 12:42:42+00:00,78.00,F,2023-12-03 13:23:00+00:00,...,4,13268,33.77400,Swim,Lucky Brand Mumbai Magic Halter Tankini Top,Lucky Brand,78.00,Women,951CB7FCF08241D659513D4E84ACDFAA,5
172080,51223,35222,28215,8039,138232,Processing,2022-08-28 12:18:49+00:00,169.00,F,2022-08-24 13:23:00+00:00,...,2,8039,107.31500,Clothing Sets,Ulla Popken Plus Size Floral 3-Piece Pant Set,Ulla Popken,169.00,Women,0919B5C38396C3F0C41F1112D538E42C,1


In [68]:
orders_products.shape

(181759, 23)

In [69]:
orders_products.columns

Index(['id_orderedproducts', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'status_item', 'created_at_item', 'sale_price',
       'gender', 'created_at_order', 'returned_at_order', 'shipped_at_order',
       'delivered_at_order', 'num_of_item', 'id_product', 'cost', 'category',
       'name', 'brand', 'retail_price', 'department', 'sku',
       'distribution_center_id'],
      dtype='object')

In [70]:
orders_products.dtypes

id_orderedproducts          int64
order_id                    int64
user_id                     int64
product_id                  int64
inventory_item_id           int64
status_item                object
created_at_item            object
sale_price                float64
gender                     object
created_at_order           object
returned_at_order          object
shipped_at_order           object
delivered_at_order         object
num_of_item                 int64
id_product                  int64
cost                      float64
category                   object
name                       object
brand                      object
retail_price              float64
department                 object
sku                        object
distribution_center_id      int64
dtype: object

# dropping columns

In [71]:
orders_products=orders_products.drop(columns=['id_product'])

# PRODUCT,ORDERS &INVENTORY (MERGE)

In [72]:
inventory_items.shape

(490705, 12)

In [73]:
490705-181759

308946

In [74]:
inventory_items.head()

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,67971,13844,2022-07-02 07:09:20+00:00,2022-07-24 06:33:20+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
1,67972,13844,2023-12-20 03:28:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
2,67973,13844,2023-06-04 02:53:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
3,72863,13844,2021-10-16 22:58:52+00:00,2021-11-22 02:19:52+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
4,72864,13844,2021-08-07 16:33:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7


In [75]:

inventory_items['created_at'] = pd.to_datetime(inventory_items['created_at'])
inventory_items['sold_at'] = pd.to_datetime(inventory_items['sold_at'])
grouped_df = inventory_items.groupby('product_id').agg({
    'created_at': lambda x: x.tolist(),
    'sold_at': lambda x: x.tolist(),
    'cost': 'first',  # Example: You can adjust this for other columns
    'product_category': 'first',
    'product_name': 'first',
    'product_brand': 'first',
    'product_retail_price': 'first',
    'product_department': 'first',
    'product_sku': 'first',
    'product_distribution_center_id': 'first',
}).reset_index()

grouped_df['count'] = inventory_items['product_id'].value_counts().tolist()
grouped_df.head()





,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id,count
0,1,"[2022-07-22 03:46:50+00:00, 2023-08-07 13:26:0...","[2022-08-03 23:10:50+00:00, NaT, 2023-08-21 14...",27.0480,Tops & Tees,Seven7 Women's Long Sleeve Stripe Belted Top,Seven7,49.0,Women,C4CA4238A0B923820DCC509A6F75849B,1,58
1,2,"[2021-03-02 23:58:57+00:00, 2022-04-23 01:51:0...","[2021-04-27 01:56:57+00:00, NaT, 2024-01-05 21...",36.6265,Tops & Tees,Calvin Klein Women's MSY Crew Neck Roll Sleeve,Calvin Klein,69.5,Women,C81E728D9D4C2F636F067F89CC14862C,3,53
2,3,"[2023-10-04 01:14:07+00:00, 2022-05-31 01:27:0...","[2023-11-27 14:19:07+00:00, NaT, 2020-01-05 23...",41.0050,Tops & Tees,Calvin Klein Jeans Women's Solid Flyaway Shirt,Calvin Klein Jeans,69.5,Women,ECCBC87E4B5CE2FE28308FD9F2A7BAF3,8,52
3,4,"[2021-09-21 19:47:57+00:00, 2021-07-04 15:34:0...","[2021-11-04 10:05:57+00:00, NaT, 2023-04-16 00...",57.1320,Tops & Tees,Bailey 44 Women's Undertow Top,Bailey 44,108.0,Women,A87FF679A2F3E71D9181A67B7542122C,6,51
4,5,"[2019-10-14 20:43:50+00:00, 2021-03-28 12:28:0...","[2019-11-20 10:05:50+00:00, NaT, 2023-11-04 22...",52.3580,Tops & Tees,Anne Klein Women's Plus-Size Button Front Blouse,Anne Klein,94.0,Women,E4DA3B7FBBCE2345D7772B0674A318D5,2,51


In [76]:
grouped_df.loc[grouped_df['product_id']==13844]

,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id,count
13786,13844,"[2022-07-02 07:09:20+00:00, 2023-12-20 03:28:0...","[2022-07-24 06:33:20+00:00, NaT, NaT, 2021-11-...",2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7,17


# FINAL ORDERS & USERS DATA (MERGE)

In [77]:
users.shape

(100000, 15)

In [78]:
users.head()

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,457,Timothy,Bush,timothybush@example.net,65,M,Acre,87620 Johnson Hills,69917-400,Rio Branco,Brasil,-9.94557,-67.83561,Search,2022-07-19 13:51:00+00:00
1,6578,Elizabeth,Martinez,elizabethmartinez@example.com,34,F,Acre,1705 Nielsen Land,69917-400,Rio Branco,Brasil,-9.94557,-67.83561,Search,2023-11-08 18:49:00+00:00
2,36280,Christopher,Mendoza,christophermendoza@example.net,13,M,Acre,125 Turner Isle Apt. 264,69917-400,Rio Branco,Brasil,-9.94557,-67.83561,Email,2019-08-24 06:10:00+00:00
3,60193,Jimmy,Conner,jimmyconner@example.com,64,M,Acre,0966 Jose Branch Apt. 008,69917-400,Rio Branco,Brasil,-9.94557,-67.83561,Search,2020-02-15 11:26:00+00:00
4,64231,Natasha,Wilson,natashawilson@example.net,25,F,Acre,20798 Phillip Trail Apt. 392,69917-400,Rio Branco,Brasil,-9.94557,-67.83561,Search,2020-03-13 06:45:00+00:00


In [79]:
orders_users_data = pd.merge(orders_products, users, 
                            left_on=['user_id'], 
                            right_on=['id'], 
                            how='left', 
                            suffixes=('_orders', '_user'))
orders_users_data.columns

Index(['id_orderedproducts', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'status_item', 'created_at_item', 'sale_price',
       'gender_orders', 'created_at_order', 'returned_at_order',
       'shipped_at_order', 'delivered_at_order', 'num_of_item', 'cost',
       'category', 'name', 'brand', 'retail_price', 'department', 'sku',
       'distribution_center_id', 'id', 'first_name', 'last_name', 'email',
       'age', 'gender_user', 'state', 'street_address', 'postal_code', 'city',
       'country', 'latitude', 'longitude', 'traffic_source', 'created_at'],
      dtype='object')

In [80]:
orders_users_data.loc[orders_users_data['user_id']==28215]

,id_orderedproducts,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,sale_price,gender_orders,created_at_order,...,gender_user,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,0.02,F,2023-12-03 13:23:00+00:00,...,F,Texas,7187 Michael Forge Apt. 777,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00
25212,51226,35223,28215,10641,138242,Complete,2023-12-06 10:12:51+00:00,16.00,F,2023-12-03 13:23:00+00:00,...,F,Texas,7187 Michael Forge Apt. 777,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00
33377,51222,35222,28215,7101,138230,Processing,2022-08-27 11:09:05+00:00,19.95,F,2022-08-24 13:23:00+00:00,...,F,Texas,7187 Michael Forge Apt. 777,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00
34789,51227,35223,28215,5268,138245,Complete,2023-12-07 10:02:18+00:00,19.99,F,2023-12-03 13:23:00+00:00,...,F,Texas,7187 Michael Forge Apt. 777,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00
141948,51225,35223,28215,13268,138240,Complete,2023-12-04 12:42:42+00:00,78.00,F,2023-12-03 13:23:00+00:00,...,F,Texas,7187 Michael Forge Apt. 777,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00
172080,51223,35222,28215,8039,138232,Processing,2022-08-28 12:18:49+00:00,169.00,F,2022-08-24 13:23:00+00:00,...,F,Texas,7187 Michael Forge Apt. 777,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00


In [81]:
orders_users_data.shape

(181759, 37)

In [82]:
orders_users_data.columns

Index(['id_orderedproducts', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'status_item', 'created_at_item', 'sale_price',
       'gender_orders', 'created_at_order', 'returned_at_order',
       'shipped_at_order', 'delivered_at_order', 'num_of_item', 'cost',
       'category', 'name', 'brand', 'retail_price', 'department', 'sku',
       'distribution_center_id', 'id', 'first_name', 'last_name', 'email',
       'age', 'gender_user', 'state', 'street_address', 'postal_code', 'city',
       'country', 'latitude', 'longitude', 'traffic_source', 'created_at'],
      dtype='object')

# DROPPING COLUMNS¶

In [83]:
orders_users_data=orders_users_data.drop(columns=['gender_orders'])
orders_users_data.rename(columns={'gender_user': 'gender',
                                         'created_at':'created_at_user'}, inplace=True)
orders_users_data.columns

Index(['id_orderedproducts', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'status_item', 'created_at_item', 'sale_price',
       'created_at_order', 'returned_at_order', 'shipped_at_order',
       'delivered_at_order', 'num_of_item', 'cost', 'category', 'name',
       'brand', 'retail_price', 'department', 'sku', 'distribution_center_id',
       'id', 'first_name', 'last_name', 'email', 'age', 'gender', 'state',
       'street_address', 'postal_code', 'city', 'country', 'latitude',
       'longitude', 'traffic_source', 'created_at_user'],
      dtype='object')

# DISTRIBUTION AND ORDERS(MERGE)

In [84]:
dist_centers.shape

(10, 4)

In [85]:
dist_centers.head()

,id,name,latitude,longitude
0,1,Memphis TN,35.1174,-89.9711
1,2,Chicago IL,41.8369,-87.6847
2,3,Houston TX,29.7604,-95.3698
3,4,Los Angeles CA,34.0500,-118.2500
4,5,New Orleans LA,29.9500,-90.0667


In [86]:
dist_centers.rename(columns={'name': 'dist_center_name'}, inplace=True)

In [87]:
orders_users_data['distribution_center_id'].unique()

array([ 1,  4,  8, 10,  5,  2,  7,  3,  9,  6])

In [88]:
dist_centers['id'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [89]:
orders_users_data=pd.merge(orders_users_data,dist_centers,left_on='distribution_center_id',
              right_on='id',how='left',suffixes=('_user','_dist'))

In [90]:
orders_users_data.columns

Index(['id_orderedproducts', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'status_item', 'created_at_item', 'sale_price',
       'created_at_order', 'returned_at_order', 'shipped_at_order',
       'delivered_at_order', 'num_of_item', 'cost', 'category', 'name',
       'brand', 'retail_price', 'department', 'sku', 'distribution_center_id',
       'id_user', 'first_name', 'last_name', 'email', 'age', 'gender', 'state',
       'street_address', 'postal_code', 'city', 'country', 'latitude_user',
       'longitude_user', 'traffic_source', 'created_at_user', 'id_dist',
       'dist_center_name', 'latitude_dist', 'longitude_dist'],
      dtype='object')

In [91]:
orders_users_data.shape

(181759, 40)

# events and all data

In [92]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2431963 entries, 0 to 2431962
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int64  
 1   user_id          float64
 2   sequence_number  int64  
 3   session_id       object 
 4   created_at       object 
 5   ip_address       object 
 6   city             object 
 7   state            object 
 8   postal_code      object 
 9   browser          object 
 10  traffic_source   object 
 11  uri              object 
 12  event_type       object 
dtypes: float64(1), int64(2), object(10)
memory usage: 241.2+ MB


In [93]:
events

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2198523,NaN,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,1773216,NaN,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,2380515,NaN,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,2250597,NaN,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,1834446,NaN,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431958,822590,62906.0,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase
2431959,43178,3333.0,5,e472549e-273e-402e-ae5f-38ece25699c2,2023-09-26 10:25:28+00:00,97.78.106.249,Orrell,England,WN5,IE,Email,/purchase,purchase
2431960,553230,42386.0,10,6a7ba027-2af2-46a3-ae05-294a264840cf,2023-07-31 15:11:10+00:00,147.252.137.75,Orrell,England,WN5,Chrome,Facebook,/purchase,purchase
2431961,1015190,77546.0,5,29e5960c-214b-456e-8c38-fe6eb8ff1ec8,2024-01-02 14:26:38+00:00,112.209.120.82,Wolverhampton,England,WV6,Chrome,Adwords,/purchase,purchase


In [94]:
events.loc[events['user_id']==28215]

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
36991,368859,28215.0,10,b673596a-974a-4945-967c-8782f47eb322,2023-12-03 12:36:49+00:00,2.66.160.231,Longview,Texas,75604,Chrome,Email,/department/women/category/swim/brand/luckybrand,department
61828,368866,28215.0,4,72ff84ee-8b4d-46fd-9e96-10ea46038f36,2023-12-03 10:01:36+00:00,10.170.63.31,Longview,Texas,75604,Safari,Email,/department/women/category/intimates/brand/mai...,department
111561,368823,28215.0,1,3fe7048c-c4c2-45f1-80da-070d0211213e,2022-08-24 10:58:28+00:00,53.161.193.4,Longview,Texas,75604,Chrome,Email,/department/women/category/skirts/brand/finebr...,department
170069,368886,28215.0,11,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 10:00:11+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/product/5268,product
185833,368876,28215.0,1,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 09:46:02+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/department/women/category/pants&capris/brand/...,department
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205365,368854,28215.0,5,b673596a-974a-4945-967c-8782f47eb322,2023-12-03 12:31:27+00:00,2.66.160.231,Longview,Texas,75604,Chrome,Email,/product/13268,product
2245786,368837,28215.0,1,91db3046-f56f-44a2-bedf-a63d9c708ad4,2023-12-03 13:08:57+00:00,182.136.87.218,Longview,Texas,75604,Safari,Email,/department/women/category/accessories/brand/m...,department
2289250,368878,28215.0,3,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 09:47:33+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/cart,cart
2304375,368860,28215.0,11,b673596a-974a-4945-967c-8782f47eb322,2023-12-03 12:38:48+00:00,2.66.160.231,Longview,Texas,75604,Chrome,Email,/product/13268,product


In [95]:
events['event_type'].value_counts()

product       845607
cart          595994
department    595323
purchase      181759
cancel        125568
home           87712
Name: event_type, dtype: int64

In [96]:
events.head()

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2198523,NaN,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,1773216,NaN,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,2380515,NaN,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,2250597,NaN,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,1834446,NaN,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel


In [97]:
events.rename(columns={'user_id': 'useeeeeer'}, inplace=True)

In [98]:
events['useeeeeer'].nunique()

80044

In [99]:
100000-80044

19956

In [100]:
# Assuming you want to drop rows with NaN values in the 'city_event' column
events = events.dropna(subset=['useeeeeer'])

# Print or display the result
events.shape


(1306292, 13)

In [101]:
fin=pd.merge(orders_users_data,events,left_on='user_id',right_on='useeeeeer',how='inner',
            suffixes=('_user','_event'))

In [102]:
fin.columns

Index(['id_orderedproducts', 'order_id', 'user_id', 'product_id',
       'inventory_item_id', 'status_item', 'created_at_item', 'sale_price',
       'created_at_order', 'returned_at_order', 'shipped_at_order',
       'delivered_at_order', 'num_of_item', 'cost', 'category', 'name',
       'brand', 'retail_price', 'department', 'sku', 'distribution_center_id',
       'id_user', 'first_name', 'last_name', 'email', 'age', 'gender',
       'state_user', 'street_address', 'postal_code_user', 'city_user',
       'country', 'latitude_user', 'longitude_user', 'traffic_source_user',
       'created_at_user', 'id_dist', 'dist_center_name', 'latitude_dist',
       'longitude_dist', 'id', 'useeeeeer', 'sequence_number', 'session_id',
       'created_at', 'ip_address', 'city_event', 'state_event',
       'postal_code_event', 'browser', 'traffic_source_event', 'uri',
       'event_type'],
      dtype='object')

In [103]:
fin.shape

(4986929, 53)

In [104]:
df1=fin.drop(columns=['city_event', 'state_event', 'postal_code_event','useeeeeer','ip_address','session_id','id','id_orderedproducts','uri','created_at'])
df1.rename(columns={'city_user': 'city','state_user':'state','postal_code_user':'postal_code'}, inplace=True)

In [105]:
df1=df1.drop_duplicates()

In [106]:
df1[(df1['product_id'] == 14235) & (df1['user_id']==28215)&(df1['sequence_number']==1)]

,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,sale_price,created_at_order,returned_at_order,shipped_at_order,...,traffic_source_user,created_at_user,id_dist,dist_center_name,latitude_dist,longitude_dist,sequence_number,browser,traffic_source_event,event_type
127,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,...,Organic,2019-06-22 13:23:00+00:00,1,Memphis TN,35.1174,-89.9711,1,Chrome,Email,department
129,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,...,Organic,2019-06-22 13:23:00+00:00,1,Memphis TN,35.1174,-89.9711,1,Chrome,YouTube,department
174,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,...,Organic,2019-06-22 13:23:00+00:00,1,Memphis TN,35.1174,-89.9711,1,Safari,Email,department


In [107]:
df1.to_csv('all_events.csv')

# dropping columns

In [299]:
fin.columns

Index(['id_orderedproducts', 'order_id', 'user_id', 'inventory_item_id',
       'status_item', 'created_at_item', 'sale_price', 'created_at_order',
       'returned_at_order', 'shipped_at_order', 'delivered_at_order',
       'num_of_item', 'id_orders', 'product_id', 'created_at_product',
       'sold_at', 'product_cost', 'product_category', 'product_name',
       'product_brand', 'product_retail_price', 'product_department',
       'product_sku', 'product_distribution_center_id', 'first_name',
       'last_name', 'email', 'age', 'gender', 'state', 'street_address',
       'postal_code', 'city', 'country', 'latitude_user', 'longitude_user',
       'traffic_source_user', 'created_at_user', 'dist_center_name',
       'latitude_dist', 'longitude_dist', 'id', 'sequence_number',
       'session_id', 'created_at', 'ip_address', 'browser',
       'traffic_source_event', 'uri', 'event_type'],
      dtype='object')

In [151]:
fin.to_csv("Final_Looker_Dataset.csv")

In [164]:
pd.set_option('display.max_columns', None)
fin.loc[fin['user_id']== 28215]

,id_orderedproducts,order_id,user_id,inventory_item_id,status_item,created_at_item,sale_price,created_at_order,returned_at_order,shipped_at_order,delivered_at_order,num_of_item,id_orders,product_id,created_at_product,sold_at,product_cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id,first_name,last_name,email,age,gender,street_address,country,latitude_user,longitude_user,traffic_source_event,created_at_user,dist_center_name,latitude_dist,longitude_dist,id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source_user,uri,event_type
35,51224,35223,28215,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4,138236,14235,2020-04-12 14:36:00+00:00,NaN,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,Rebecca,Mcgee,rebeccamcgee@example.org,49,F,7187 Michael Forge Apt. 777,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Memphis TN,35.1174,-89.9711,368859,10,b673596a-974a-4945-967c-8782f47eb322,2023-12-03 12:36:49+00:00,2.66.160.231,Longview,Texas,75604,Chrome,Email,/department/women/category/swim/brand/luckybrand,department
36,51224,35223,28215,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4,138236,14235,2020-04-12 14:36:00+00:00,NaN,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,Rebecca,Mcgee,rebeccamcgee@example.org,49,F,7187 Michael Forge Apt. 777,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Memphis TN,35.1174,-89.9711,368866,4,72ff84ee-8b4d-46fd-9e96-10ea46038f36,2023-12-03 10:01:36+00:00,10.170.63.31,Longview,Texas,75604,Safari,Email,/department/women/category/intimates/brand/mai...,department
37,51224,35223,28215,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4,138236,14235,2020-04-12 14:36:00+00:00,NaN,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,Rebecca,Mcgee,rebeccamcgee@example.org,49,F,7187 Michael Forge Apt. 777,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Memphis TN,35.1174,-89.9711,368823,1,3fe7048c-c4c2-45f1-80da-070d0211213e,2022-08-24 10:58:28+00:00,53.161.193.4,Longview,Texas,75604,Chrome,Email,/department/women/category/skirts/brand/finebr...,department
38,51224,35223,28215,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4,138236,14235,2020-04-12 14:36:00+00:00,NaN,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,Rebecca,Mcgee,rebeccamcgee@example.org,49,F,7187 Michael Forge Apt. 777,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Memphis TN,35.1174,-89.9711,368886,11,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 10:00:11+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/product/5268,product
39,51224,35223,28215,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4,138236,14235,2020-04-12 14:36:00+00:00,NaN,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,Rebecca,Mcgee,rebeccamcgee@example.org,49,F,7187 Michael Forge Apt. 777,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Memphis TN,35.1174,-89.9711,368876,1,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 09:46:02+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/department/women/category/pants&capris/brand/...,department
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [154]:
pd.set_option('display.max_columns', None)
events.loc[events['useeeeeer']==28215]

,id,useeeeeer,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
36991,368859,28215.0,10,b673596a-974a-4945-967c-8782f47eb322,2023-12-03 12:36:49+00:00,2.66.160.231,Longview,Texas,75604,Chrome,Email,/department/women/category/swim/brand/luckybrand,department
61828,368866,28215.0,4,72ff84ee-8b4d-46fd-9e96-10ea46038f36,2023-12-03 10:01:36+00:00,10.170.63.31,Longview,Texas,75604,Safari,Email,/department/women/category/intimates/brand/mai...,department
111561,368823,28215.0,1,3fe7048c-c4c2-45f1-80da-070d0211213e,2022-08-24 10:58:28+00:00,53.161.193.4,Longview,Texas,75604,Chrome,Email,/department/women/category/skirts/brand/finebr...,department
170069,368886,28215.0,11,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 10:00:11+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/product/5268,product
185833,368876,28215.0,1,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 09:46:02+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/department/women/category/pants&capris/brand/...,department
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205365,368854,28215.0,5,b673596a-974a-4945-967c-8782f47eb322,2023-12-03 12:31:27+00:00,2.66.160.231,Longview,Texas,75604,Chrome,Email,/product/13268,product
2245786,368837,28215.0,1,91db3046-f56f-44a2-bedf-a63d9c708ad4,2023-12-03 13:08:57+00:00,182.136.87.218,Longview,Texas,75604,Safari,Email,/department/women/category/accessories/brand/m...,department
2289250,368878,28215.0,3,70200224-28ab-41da-8eee-be031ba1c391,2023-12-03 09:47:33+00:00,13.180.141.96,Longview,Texas,75604,Chrome,YouTube,/cart,cart
2304375,368860,28215.0,11,b673596a-974a-4945-967c-8782f47eb322,2023-12-03 12:38:48+00:00,2.66.160.231,Longview,Texas,75604,Chrome,Email,/product/13268,product


In [156]:
orders_users_data.loc[orders_users_data['user_id']==28215]

,id_orderedproducts,order_id,user_id,inventory_item_id,status_item,created_at_item,sale_price,created_at_order,returned_at_order,shipped_at_order,...,postal_code,city,country,latitude_user,longitude_user,traffic_source,created_at_user,dist_center_name,latitude_dist,longitude_dist
2,51224,35223,28215,138236,Complete,2023-12-05 13:25:30+00:00,0.02,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,...,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Memphis TN,35.1174,-89.9711
25212,51226,35223,28215,138242,Complete,2023-12-06 10:12:51+00:00,16.00,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,...,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Charleston SC,32.7833,-79.9333
33377,51222,35222,28215,138230,Processing,2022-08-27 11:09:05+00:00,19.95,2022-08-24 13:23:00+00:00,NaN,NaN,...,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Philadelphia PA,39.9500,-75.1667
34789,51227,35223,28215,138245,Complete,2023-12-07 10:02:18+00:00,19.99,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,...,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Houston TX,29.7604,-95.3698
141948,51225,35223,28215,138240,Complete,2023-12-04 12:42:42+00:00,78.00,2023-12-03 13:23:00+00:00,NaN,2023-12-06 01:20:00+00:00,...,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,New Orleans LA,29.9500,-90.0667
172080,51223,35222,28215,138232,Processing,2022-08-28 12:18:49+00:00,169.00,2022-08-24 13:23:00+00:00,NaN,NaN,...,75604,Longview,United States,32.51243,-94.82549,Organic,2019-06-22 13:23:00+00:00,Memphis TN,35.1174,-89.9711


In [157]:
orders_users_data.to_csv("orders_users_data.csv")